In [1]:
import cv2
import numpy as np

# 예제 8.1 cv2.preCornerDetect() 코너점 검출

In [2]:
def findLocalMaxima(src):
    kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize = (11,11)) # 1로 채워진 11*11 크기의 배열
    dilate = cv2.dilate(src, kernel) # 팽창: 전경이 되는 이미지의 경계 부분을 팽창 -> 전경 이미지가 굵어지는 효과
    localMax = (src == dilate) # 픽셀값의 변화가 없는 곳(배경이나 전경 이미지)은 1, 변화가 생긴 곳(전경 이미지 굻어지는 부분)은 0
    # print(localMax)
    erode = cv2.erode(src, kernel) # 침식: 전경이 되는 이미지의 경계 부분을 침식시켜 배경 이미지로 전환 -> 전경 이미지가 가늘어지는 효과
    localMax2 = src > erode
    # print(localMax2)
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0,1]] = points[:,[1,0]]
    return points

<details>
<summary>erosion 이미지</summary>
<div markdown="1">
<img src="https://user-images.githubusercontent.com/84698896/213682801-d5c866e2-2241-4387-be50-8f6f843cd443.png"  width="20%" height="20%"/>
</div>
</details>

### <span style='background-color:#fff5b1'>cv2.preCornerDetect(src, ksize)</span>
src에서 코너점 검출을 위한 특징맵 dst를 sobel 미분 연산자를 이용하여 계산

In [3]:
src = np.zeros(shape = (512,512,3), dtype = np.uint8) + 255
cv2.rectangle(src, (100,100), (300,300), (0,0,0), -1)
cv2.rectangle(src, (200,200), (400,400), (0,0,0), -1)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
corners.shape

(8, 2)

In [4]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 5, (0,0,255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 8.2 코너점 검출: cornerEigenValsAndVecs()
### <span style='background-color:#fff5b1'>cv2.cornerDigenValsAndVecs(src, blockSize, ksize)</span>
src에서 각 화소의 고유값과 고유 벡터를 6-채널 dst에 계산 \
이웃: blockSize * blockSize

In [5]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize = 5, ksize = 3)
eigen = cv2.split(res)
res.shape

(512, 512, 6)

In [6]:
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

In [7]:
corners = np.argwhere(eigen[1] > T)
corners[:,[0,1]] = corners[:,[1,0]]
len(corners)

8

In [8]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 5, (0,0,255), 2)

cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 8.3 코너점 검출 1: cv2.cornerMinEigenVal()
### <span style='background-color:#fff5b1'>cv2.cornerMinEigenVal(src, blockSize = 5)</span>
src에서 각 화소의 최소 고유값을 dst로 반환

In [9]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
eigen.shape

(512, 512)

In [10]:
T = 0.2
corners = np.argwhere(eigen > T)
corners[:,[0,1]] = corners[:,[1,0]]
len(corners)

8

In [11]:
dst = src.copy()
for x, y in corners:
    cv2.circle(dst, (x,y), 3, (0,0,255), 2)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 8.4 코너점 검출 2: cv2.cornerHarris(), cv2.cornerSubPix()
### <span style='background-color:#fff5b1'>cv2.cornerSubPix(src, blockSize, ksize, k)</span>
src에서 각 화소의 Harris 반응값을 계산하여 dst에 반환

In [12]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize = 5, ksize = 3, k = 0.01)
ret, res = cv2.threshold(np.abs(res), 0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255,  cv2.NORM_MINMAX, dtype = cv2.CV_8U)

corners = findLocalMaxima(res)
corners

array([[101, 101],
       [299, 101],
       [302, 198],
       [399, 201],
       [101, 299],
       [198, 302],
       [201, 399],
       [399, 399]])

### <span style='background-color:#fff5b1'>cv2.cornerSubPix(src, corners, winSize, zeroZone, criteria)</span>
src에서 검출된 코너점 corners를 입력하여 코너점의 위치를 부화소 수준으로 다시 계산하여 반환 \
**winSize** 탐색 영역의 크기 \
**zeroZone** winSize 내에서 zeroZone 영역을 마스크 처리하여 탐색 영역에서 계산하지 않음

In [13]:
corners = corners.astype(np.float32, order = 'C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.01)

corners2 = cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), term_crit)
corners2

array([[ 99.55921,  99.55921],
       [300.4408 ,  99.55921],
       [300.5592 , 199.44078],
       [400.4408 , 199.55922],
       [ 99.55921, 300.4408 ],
       [199.44078, 300.5592 ],
       [199.55922, 400.4408 ],
       [400.4408 , 400.4408 ]], dtype=float32)

In [14]:
dst = src.copy()
for x, y in np.int32(corners2):
    cv2.circle(dst, (x,y), 3, (0,0,255), 2)
cv2.imshow('res8', res8)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 8.5 코너점 검출 2: cv2.cornerHarris(). cv2.cornerSubPix()</span>

In [15]:
res = cv2.cornerHarris(gray, blockSize = 5, ksize = 3, k = 0.01)

In [16]:
res = cv2.dilate(res, None)
ret, res = cv2.threshold(res, 0.01 * res.max(), 255, cv2.THRESH_BINARY)
res8 = np.uint8(res)

In [17]:
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(res8)
centroids = np.float32(centroids)
centroids.shape

(9, 2)

In [18]:
tern_crit = (cv2.TermCriteria_MAX_ITER + cv2.TermCriteria_EPS, 10, 0.001)
corners = cv2.cornerSubPix(gray, centroids, (5,5), (-1,-1), term_crit)
corners

array([[255.50824 , 255.50824 ],
       [ 99.557724,  99.557724],
       [300.44226 ,  99.557724],
       [300.55774 , 199.44228 ],
       [400.44226 , 199.55772 ],
       [ 99.557724, 300.44226 ],
       [199.44228 , 300.55774 ],
       [199.55772 , 400.44226 ],
       [400.44226 , 400.44226 ]], dtype=float32)

In [19]:
corners = np.round(corners)
dst = src.copy()
for x, y in corners[1:]:
    cv2.circle(dst, (int(x), int(y)), 5, (0,0,255), 2)

cv2.imshow('res8', res)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 8.6 cv2.goodFeaturesToTrack() 코너점 검출
### <span style='background-color:#fff5b1'>cv2.goodFeaturesToTrack(src, maxCorners, qulityLevel, minDistance, corners, mask, blockSize, useHarrisDetector, k)</span>
src에서 추적하기 좋은 강한 코너점을 검출하여 반환 \
**maxCorners** 최대 코너점 개수 \
**qualityLevel** 최소 코너점읠 quality를 결정하는 값 \
**minDistance** 코너점들 사이의 최소 거리 \
**mask** 코너점이 검출될 영역 \
**useHarrisDetector** cv2.cornerHarris : cv2.cornerMinEigenVal() \
**k** Harris 검출에 사용되는 상수

In [20]:
K = 5
corners = cv2.goodFeaturesToTrack(gray, maxCorners=K, qualityLevel=0.05, minDistance=10)
corners

array([[[400., 400.]],

       [[200., 400.]],

       [[199., 301.]],

       [[100., 300.]],

       [[400., 200.]]], dtype=float32)

In [21]:
corners2 = cv2.goodFeaturesToTrack(gray, maxCorners=K, qualityLevel=0.05, minDistance=120, useHarrisDetector=True, k=0.04)
corners2

array([[[400., 400.]],

       [[200., 400.]],

       [[100., 300.]],

       [[400., 200.]],

       [[300., 100.]]], dtype=float32)

In [22]:
dst = src.copy()

corners = corners.reshape(-1,2)
for x, y in corners:
    cv2.circle(dst, (int(x), int(y)), 5, (0,0,255), -1)

corners2 = corners2.reshape(-1,2)
for x, y in corners2:
    cv2.circle(dst, (int(x), int(y)), 5, (255,0,0), 2)

In [23]:
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()